In [ ]:
import pdfplumber
import pandas as pd
import json
import re
from unicodedata import normalize


def remove_tildes(string: str) -> str:
    string = re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
        normalize("NFD", string), 0, re.I
    )
    string = normalize("NFC", string)
    return string


In [2]:
with pdfplumber.open("./raw/Tabla-Códigos-Dane.pdf") as pdf:
    page = pdf.pages[0]
    table = page.extract_table()

    for page in pdf.pages[1:]:
        table.extend(page.extract_table())

df = pd.DataFrame(table[1:], columns=table[0])
df

,Código\nDepartamento,Nombre\nDepartamento,Código\nMunicipio,Nombre Municipio
0,05,ANTIOQUIA,001,MEDELLIN
1,05,ANTIOQUIA,002,ABEJORRAL
2,05,ANTIOQUIA,004,ABRIAQUI
3,05,ANTIOQUIA,021,ALEJANDRIA
4,05,ANTIOQUIA,030,AMAGA
...,...,...,...,...
1115,97,VAUPES,889,YAVARATE
1116,99,VICHADA,001,PUERTO CARREÑO
1117,99,VICHADA,524,LA PRIMAVERA
1118,99,VICHADA,624,SANTA ROSALIA


In [41]:
# Don't know what would be better, so I created both xd
data1 = {}  # Save code as key, then department name (or municipality) 
data2 = {}  # Save department name as key, then code

for dpto in df["Nombre\nDepartamento"].unique():
    sub_df = df.loc[df["Nombre\nDepartamento"] == dpto].reset_index()
    code_dpto = sub_df["Código\nDepartamento"].unique()[0]
    arr = sub_df[["Nombre Municipio", "Código\nMunicipio"]].to_numpy()

    data1[dpto] = {
        "department": dpto,
        "municipalities": [{cod: muni for muni, cod in arr}]
    }

    data2[dpto] = {
        "code": code_dpto,
        "municipalities": [{muni: cod for muni, cod in arr}]
    }

In [42]:
with open("./processed/cods_dptos_mpios.json", "w", encoding="utf-8") as json_file1, \
    open("./processed/dpts_cods_mpios.json", "w", encoding="utf-8") as json_file2:
    
    json.dump(data1, json_file1, indent=4)
    json.dump(data2, json_file2, indent=4)